In [ ]:
import os
update_webui = True
update_extensions = True
branch = "master"
root_path = "/kaggle/working"
webui_path = os.path.join(root_path, "x1101")
config = "https://raw.githubusercontent.com/DEX-1101/SecretNAI/main/config.json"

#Kaggle
xformers = "xformers==0.0.20 triton==2.0.0"
torch = "torch==2.0.1+cu118 torchvision==0.15.2+cu118 torchaudio==2.0.2+cu118 torchtext==0.15.2 torchdata==0.6.1 --extra-index-url https://download.pytorch.org/whl/cu118"

#Colab
#xformers = "xformers==0.0.22.post7"

print("Please wait...")
!curl -OL https://cdn.discordapp.com/attachments/594353332508819457/1184540418369667233/tunnel --output-dir {root_path}
!curl -s -Lo /usr/bin/cl https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 && chmod +x /usr/bin/cl
!pip install -q git+https://github.com/DEX-1101/colablib
import time
from colablib.colored_print import cprint, print_line
from colablib.utils import py_utils
from IPython.display import clear_output
start_time    = time.time()
clear_output()

%cd {root_path}
if not os.path.exists(webui_path):
    print_line(0, color="green")
    cprint("[+] Preparing Environment", color="flat_cyan")
    cprint("    Installing requirements...", color="flat_yellow")
    !apt -y install -qq aria2 &> /dev/null
    !conda install -q -y -c conda-forge pyyaml &> /dev/null
    cprint("    Installing torch...", color="flat_yellow")
    !pip install -q {torch} -U
    cprint("    Installing xformers...", color="flat_yellow")
    !pip install -q {xformers} -U
    !wget -q https://raw.githubusercontent.com/DEX-1101/SecretNAI/main/template.txt -O {root_path}/download_list.txt
    cprint("    Done!", color="flat_yellow")
    print_line(0, color="green")
    cprint("[+] Downloading Repository...", color="flat_cyan")
    !aria2c --console-log-level=error -q -c -x 16 -s 16 -k 1M https://huggingface.co/x1101/fix/resolve/main/zip/fastrepo_ext_def.zip -o fastrepo_ext.zip
    cprint("    Unpacking...", color="flat_yellow")
    !unzip -q -o fastrepo_ext.zip 
    !mv {root_path}/home/studio-lab-user/content/x1101 {root_path}
    !git clone -q https://github.com/zanllp/sd-webui-infinite-image-browsing {webui_path}/extensions/infinite-image-browsing
    !git clone -q https://github.com/DEX-1101/sd-encrypt-image {webui_path}/extensions/encryptimg
    !git clone -q https://github.com/Haoming02/sd-webui-tabs-extension {webui_path}/extensions/tab-extension
    !git clone -q https://github.com/wcde/sd-webui-kohya-hiresfix {webui_path}/extensions/hiresfix
    !git clone -q https://github.com/NoCrypt/sd-fast-pnginfo {webui_path}/extensions/fast-p
    !rm {root_path}/fastrepo_ext.zip
    !rm -rf {root_path}/home
else:
    print_line(0, color="green")
    cprint("[+] Repository already exists. Skipping...", color="flat_cyan")  

if update_webui:
    print_line(0, color="green")
    cprint("[+] Updating Repository...", color="flat_cyan")
    %cd {webui_path}
    !git pull &> /dev/null   
    !git switch {branch}
    !git pull
    !git reset --hard
    
if update_extensions:
    print_line(0, color="green")
    cprint("[+] Updating Extensions...", color="flat_cyan")
    %cd {webui_path}/extensions
    !find . -mindepth 1 -maxdepth 1 -type d -print -exec git -C {} reset --hard \; &> /dev/null
    !find . -mindepth 1 -maxdepth 1 -type d -print -exec git -C {} pull \; &> /dev/null
    !find . -mindepth 1 -maxdepth 1 -type d -print -exec git -C {} reset --hard \;
    
!wget -q {config} -O {webui_path}/config.json
!wget -q "https://civitai.com/api/download/models/134583?type=Model&format=SafeTensor" -O {webui_path}/embeddings/negativeXL.safetensors
!wget -q "https://civitai.com/api/download/models/94057?type=Model&format=PickleTensor" -O {webui_path}/embeddings/FastNegative2.pt
elapsed_time  = py_utils.calculate_elapsed_time(start_time)
print_line(0, color="green")
cprint(f"[+] All completed within {elapsed_time}.", color="flat_cyan")

In [ ]:
#link = "https://huggingface.co/Linaqruf/animagine-xl-2.0/resolve/main/animagine-xl-2.0.safetensors"
link = "https://civitai.com/api/download/models/100675?type=Model&format=SafeTensor&size=pruned&fp=fp16"

name = "meina.safetensors"

%cd {webui_path}/models/Stable-diffusion
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {link} -o {name}

In [ ]:
pastebin_url = "https://pastebin.com/mr0v09pG"
hf_token = ""

#=================================================

#Code by Bang Furqanil
import os
import time
from colablib.utils import py_utils
from pydantic import BaseModel
from colablib.utils.py_utils import get_filename
from colablib.sd_models.downloader import aria2_download, download
from colablib.colored_print import cprint, print_line
from colablib.utils.config_utils import read_config
from colablib.utils.git_utils import clone_repo

%store -r

custom_model_url        = ""
custom_vae_url          = ""
custom_embedding_url    = ""
custom_LoRA_url         = ""
custom_extensions_url   = ""
models_dir          = os.path.join(webui_path, "models", "Stable-diffusion")
vaes_dir            = os.path.join(webui_path, "models", "VAE")
lora_dir            = os.path.join(webui_path, "models", "Lora")
embeddings_dir      = os.path.join(webui_path, "embeddings")
extensions_dir      = os.path.join(webui_path, "extensions")
download_list       = os.path.join(root_path, "download_list.txt")
class CustomDirs(BaseModel):
    url: str
    dst: str
user_header = f"Authorization: Bearer {hf_token}"
custom_dirs = {
    "model"       : CustomDirs(url=custom_model_url, dst=models_dir),
    "vae"         : CustomDirs(url=custom_vae_url, dst=vaes_dir),
    "embedding"   : CustomDirs(url=custom_embedding_url, dst=embeddings_dir),
    "lora"        : CustomDirs(url=custom_LoRA_url, dst=lora_dir),
    "extensions"  : CustomDirs(url=custom_extensions_url, dst=extensions_dir),
}

def parse_urls(filename):
    content = read_config(filename)
    lines   = content.strip().split('\n')
    result  = {}
    key     = ''
    for line in lines:
        if not line.strip():
            continue
        if line.startswith('//'):
            continue
        if line.startswith('#'):
            key = line[1:].lower()
            result[key] = []
        else:
            urls = [url.strip() for url in line.split(',') if url.strip() != '']
            result[key].extend(urls)
    return result

def get_filename(url, token=None):
    headers = {}
    if token:
        headers['Authorization'] = f'Bearer {hf_token}'

def custom_download(custom_dirs):
    for key, value in custom_dirs.items():
        urls     = value.url.split(",")  # Split the comma-separated URLs
        dst      = value.dst

        if value.url:
            print_line(0, color="green")
            cprint(f" [+] Downloading {key}.", color="flat_yellow")

        for url in urls:
            url = url.strip()  # Remove leading/trailing whitespaces from each URL
            if url != "":
                print_line(0, color="green")
                if "|" in url:
                    url, filename = map(str.strip, url.split("|"))
                    if not filename.endswith((".safetensors", ".ckpt", ".pt", "pth")):
                        filename = filename + os.path.splitext(get_filename(url))[1]
                else:
                    if not url.startswith("fuse:"):
                        filename = get_filename(url)

                if url.startswith("fuse:"):
                    fuse(url, key, dst)
                elif key == "extensions":
                    clone_repo(url, cwd=dst)
                else:
                   download(url=url, filename=filename, user_header=user_header, dst=dst, quiet=False)

def download_from_textfile(filename):
    for key, urls in parse_urls(filename).items():
        key_lower = key.lower()
        if key_lower in custom_dirs:
            if custom_dirs[key_lower].url:
                custom_dirs[key_lower].url += ',' + ','.join(urls)
            else:
                custom_dirs[key_lower].url = ','.join(urls)
        else:
            cprint(f"Warning: Category '{key}' from the file is not found in custom_dirs.", color="flat_yellow")

def custom_download_list(url):
    filename = "custom_download_list.txt"
    filepath = os.path.join(root_path, filename)
    if os.path.exists(filepath):
        os.remove(filepath)
    if 'pastebin.com' in url:
        if 'raw' not in url:
            url = url.replace('pastebin.com', 'pastebin.com/raw')
    download(url=url, filename=filename, user_header=user_header, dst=root_path, quiet=True)
    return filepath

def main():
    start_time    = time.time()
    textfile_path = download_list
    if pastebin_url:
        textfile_path = custom_download_list(pastebin_url)
    download_from_textfile(textfile_path)
    custom_download(custom_dirs)

    elapsed_time  = py_utils.calculate_elapsed_time(start_time)
    print_line(0, color="green")
    cprint(f"[+] Download completed within {elapsed_time}.", color="flat_yellow")
main()

In [ ]:
ngrok_token = ""
password = "x1101"


from colablib.colored_print import cprint, print_line
import time
try:
    start_colab
except:
    start_colab = int(time.time())-5
    
%cd {root_path}
import cloudpickle as pickle
tunnel = pickle.load(open("tunnel", "rb"), encoding="utf-8")
import re
tunnel_port= 1101
tunnel_list = [
    {
        "command": "cl tunnel --url localhost:{port}",
        "pattern": re.compile(r"[\w-]+\.trycloudflare\.com"),
    },
]
with tunnel(tunnel_port, tunnel_list):
    %cd {webui_path}
    !echo -n {start_colab} >{webui_path}/static/colabTimer.txt
    !sed -i -e 's/\["sd_model_checkpoint"\]/\["sd_model_checkpoint","sd_vae","CLIP_stop_at_last_layers"\]/g' {webui_path}/modules/shared_options.py
    !python launch.py --port=1101 --ngrok {ngrok_token}  --encrypt-pass={password} --xformers --theme dark --enable-insecure-extension-access --disable-console-progressbars --no-half-vae

/kaggle/working
Tunnel Started
Waiting 1101 to be accesible
/kaggle/working/x1101
Python 3.10.12 | packaged by conda-forge | (main, Jun 23 2023, 22:40:32) [GCC 12.3.0]
Version: v1.7.0
Commit hash: cf2772fab0af5573da775e7437e6acdca424f26e
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/1.1 MB ? eta -:--:--
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/1.1 MB ? eta -:--:--
     ━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/1.1 MB 776.6 kB/s eta 0:00:02
     ━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.1/1.1 MB 1.5 MB/s eta 0:00:01
     ━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.3/1.1 MB 2.4 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━━━ 0.7/1.1 MB 4.0 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 1.0/1.1 MB 5.2 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 4.8 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.21.12
    Uninstalling protobuf-4.21.12:
      Successfully uninstalled p

### lowres, bad anatomy, bad hands, text, error, missing fingers, extra digit, fewer digits, cropped, worst quality, low quality, normal quality, jpeg artifacts, signature, watermark, username, blurry, artist name

In [ ]:
%cd {webui_path}/extensions/sd-webui-controlnet/models/

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11f1e_sd15_tile_fp16.safetensors -o control_v11f1e_sd15_tile_fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11f1e_sd15_tile_fp16.yaml -o control_v11f1e_sd15_tile_fp16.yaml

In [ ]:
# HuggingFace Upload
token = ""

# Make Repo
repo_name = "models"
private_repo = True 

# Upload
model_repo = "x1101/models"
model_path = "/kaggle/working/x1101/models/Stable-diffusion/0.5(AAMGape) + 0.5(AnythingV5Ink_v5PrtRE).safetensors"
path_in_repo = ""
commit_message = ""

#==============================
%pip install huggingface_hub &> /dev/null
from huggingface_hub import login
from huggingface_hub import HfApi
from huggingface_hub.utils import validate_repo_id, HfHubHTTPError
from huggingface_hub import HfApi
from pathlib import Path
%store -r
api = HfApi()
login(token, add_to_git_credential=True)

api = HfApi()
user = api.whoami(token)

if repo_name:
    try:
        model_repo = user["name"] + "/" + repo_name.strip()
        validate_repo_id(model_repo)
        api.create_repo(repo_id=model_repo, private=private_repo)
        print("Repository doesn't exists, creating Repository...")
        print("Repository: ", model_repo, "created!\n")

    except HfHubHTTPError as e:
        print(f"Repository : {model_repo} exists, skipping create Repository\n")

if not commit_message:
  commit_message = "feat: upload"

def upload_model(model_paths, is_folder :bool, is_config :bool):
  path_obj = Path(model_paths)
  trained_model = path_obj.parts[-1]

  if path_in_repo:
        trained_model = path_in_repo

  if is_folder:
    print(f"Uploading {trained_model} to https://huggingface.co/"+model_repo)
    print(f"Please wait...")

    api.upload_folder(
        folder_path=model_paths,
        path_in_repo=trained_model,
        repo_id=model_repo,
        commit_message=commit_message,
        ignore_patterns=".ipynb_checkpoints"
        )
    print(f"Upload success, located at https://huggingface.co/"+model_repo+"/tree/main\n")
  else:
    print(f"Uploading {trained_model} to https://huggingface.co/"+model_repo)
    print(f"Please wait...")

    api.upload_file(
        path_or_fileobj=model_paths,
        path_in_repo=trained_model,
        repo_id=model_repo,
        commit_message=commit_message,
        )

    print(f"Upload success, located at https://huggingface.co/"+model_repo+"/blob/main/"+trained_model+"\n")

def upload():
    if model_path.endswith((".ckpt", ".safetensors", ".pt")):
      upload_model(model_path, False, False)
    else:
      upload_model(model_path, True, False)

upload()